In [29]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.contrib.slim as slim
import os
import scipy.misc
import scipy
import sklearn.datasets
import random
import itertools

MESSAGE_SIZE=40

In [30]:
import tensorflow as tf

def binarize(x):
    g = tf.get_default_graph()
    with g.gradient_override_map({"Sign": "Identity"}):
        return tf.sign(x)

In [31]:
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images
train_data = np.reshape(train_data, [-1, 28, 28, 1])

input_data = tf.placeholder("float", [None, 28, 28, 1])
resize = tf.image.resize_images(
    input_data,
    [32, 32]
)

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    train_data =  sess.run(resize, feed_dict={input_data: train_data})
    train_data = (train_data - 0.5) * 2



Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz


In [32]:
def generator(message, isTrain=True, reuse=False):
    with tf.variable_scope('generator', reuse=reuse):
        x = tf.reshape(tf.contrib.layers.flatten(tf.cast(message, tf.float32)), [-1, 1, 1, MESSAGE_SIZE])
        conv1 = tf.layers.conv2d_transpose(x, 512, [4, 4], strides=(2, 2), padding='valid')
        lrelu1 = tf.nn.leaky_relu(tf.layers.batch_normalization(conv1, training=isTrain))

        conv2 = tf.layers.conv2d_transpose(lrelu1, 256, [4, 4], strides=(2, 2), padding='same')
        lrelu2 = tf.nn.leaky_relu(tf.layers.batch_normalization(conv2, training=isTrain))

        conv3 = tf.layers.conv2d_transpose(lrelu2, 128, [4, 4], strides=(2, 2), padding='same')
        lrelu3 = tf.nn.leaky_relu(tf.layers.batch_normalization(conv3, training=isTrain))

        conv4 = tf.layers.conv2d_transpose(lrelu3, 1, [4, 4], strides=(2, 2), padding='same')
        out = tf.nn.tanh(conv4)

        return binarize(out), out

In [33]:
def decoder(input_batch, isTrain=True, reuse=False):
    with tf.variable_scope('decoder', reuse=reuse):
        conv1 = tf.layers.conv2d(input_batch, 128, [4, 4], strides=(2, 2), padding='same')
        lrelu1 = tf.nn.leaky_relu(tf.layers.batch_normalization(conv1, training=isTrain))

        conv2 = tf.layers.conv2d(lrelu1, 256, [4, 4], strides=(2, 2), padding='same')
        lrelu2 = tf.nn.leaky_relu(tf.layers.batch_normalization(conv2, training=isTrain))

        conv3 = tf.layers.conv2d(lrelu2, 512, [4, 4], strides=(2, 2), padding='same')
        lrelu3 = tf.nn.leaky_relu(tf.layers.batch_normalization(conv3, training=isTrain))
        
        dense1 = tf.layers.dense(tf.contrib.layers.flatten(lrelu3), MESSAGE_SIZE, activation=tf.nn.leaky_relu)    
#         dense1 = tf.layers.dense(tf.contrib.layers.flatten(input_batch), 50)
        out = tf.nn.tanh(dense1)
        return binarize(tf.reshape(out, shape=[-1, MESSAGE_SIZE, 1, 1]))

In [34]:
def discriminator(x, isTrain=True, reuse=False):
    with tf.variable_scope('discriminator', reuse=reuse):
        conv1 = tf.layers.conv2d(x, 128, [4, 4], strides=(2, 2), padding='same')
        lrelu1 = tf.nn.leaky_relu(tf.layers.batch_normalization(conv1, training=isTrain))

        conv2 = tf.layers.conv2d(lrelu1, 256, [4, 4], strides=(2, 2), padding='same')
        lrelu2 = tf.nn.leaky_relu(tf.layers.batch_normalization(conv2, training=isTrain))

        conv3 = tf.layers.conv2d(lrelu2, 512, [4, 4], strides=(2, 2), padding='same')
        lrelu3 = tf.nn.leaky_relu(tf.layers.batch_normalization(conv3, training=isTrain))

        conv4 = tf.layers.conv2d(lrelu3, 1, [4, 4], strides=(1, 1), padding='valid')
        out = tf.nn.sigmoid(conv4)

        return out, conv4

In [35]:
fixed_m_ = (np.random.randint(2, size=[16, MESSAGE_SIZE, 1, 1]) - 0.5) * 2
print(fixed_m_[0].T)
dec_accuracy = 0.
def show_result(epoch, iter, accuracy='0.00', path = 'result.png', binary=False):
    if binary:
        test_images = sess.run(bin_G_z, { message: fixed_m_, isTrain: False})
    else:
        test_images = sess.run(G_z, { message: fixed_m_, isTrain: False})
    size_figure_grid = 4
    fig, ax = plt.subplots(size_figure_grid, size_figure_grid, figsize=(4, 4))
    for i, j in itertools.product(range(size_figure_grid), range(size_figure_grid)):
        ax[i, j].get_xaxis().set_visible(False)
        ax[i, j].get_yaxis().set_visible(False)

    for k in range(size_figure_grid*size_figure_grid):
        i = k // size_figure_grid
        j = k % size_figure_grid
        ax[i, j].cla()
        ax[i, j].imshow(np.reshape(test_images[k], (32, 32)), cmap='gray')

    label = 'Epoch: ' + str(epoch) + ", iteration: " + str(iter) + ", decode accuracy: " + accuracy
    fig.text(0.5, 0.04, label, ha='center')

    if binary:
        plt.savefig(path + "bin.png")
    else:
        plt.savefig(path)

    plt.close()


[[[-1.  1.  1.  1. -1. -1.  1. -1.  1.  1.  1. -1. -1.  1. -1.  1.  1.
    1. -1. -1.  1. -1. -1. -1.  1. -1.  1.  1.  1.  1. -1.  1. -1. -1.
    1. -1.  1.  1. -1. -1.]]]


In [36]:
def binary(arr):
    out = np.copy(arr)
    out[arr > 0] = 1.0
    out[arr <= 0] = 0.0
    return out

def minus(one, two):
    arr = binary(one) - binary(two)
    arr[arr != 0] = 1
    return sum(arr)

def test_message():
    m_ = (np.random.randint(2, size=[batch_size, MESSAGE_SIZE, 1, 1]) - 0.5) * 2
    output_m = decoder(bin_G_z, False, True)
    
    out_m_ = sess.run(output_m, { message: m_, isTrain: False})
    
    out_m_ = np.reshape(out_m_, [-1, MESSAGE_SIZE])
    m_ = np.reshape(m_, [-1, MESSAGE_SIZE])
    out = 0.
    for i in range(0, len(m_)):
        out += minus(m_[i], out_m_[i])
        
    allD = len(m_) * len(m_[0])
    allD = 1. * allD
    acc = (1. - out/allD) * 100
    print("decode acc: " + str(acc) + "%")
    return acc



In [ ]:
# reset - because of jupyter
tf.reset_default_graph()

# variables
batch_size = 100
lr = 0.0001

# placeholders
x = tf.placeholder(tf.float32, shape=(None, 32, 32, 1))

# message
message = tf.placeholder(tf.float32, shape=(None, MESSAGE_SIZE, 1, 1))
isTrain = tf.placeholder(dtype=tf.bool)

# generator - fake
bin_G_z, G_z = generator(message, isTrain)
DEC_g = decoder(bin_G_z, isTrain)

# discriminator - real
D_real, D_real_logits = discriminator(x, isTrain)

# discriminator - fake
D_fake, D_fake_logits = discriminator(G_z, isTrain, reuse=True)

# loses
# disc
D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_real_logits, labels=tf.ones([batch_size, 1, 1, 1])))
D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake_logits, labels=tf.zeros([batch_size, 1, 1, 1])))
D_loss = D_loss_real + D_loss_fake

# gen
DEC_loss = tf.losses.mean_squared_error(DEC_g, message)
G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake_logits, labels=tf.ones([batch_size, 1, 1, 1])))

# trainable variables
T_vars = tf.trainable_variables()
D_vars = [var for var in T_vars if var.name.startswith('discriminator')]
G_vars = [var for var in T_vars if var.name.startswith('generator') or var.name.startswith('decoder')]

# optimizers
with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
    D_optim = tf.train.AdamOptimizer(lr, beta1=0.4).minimize(D_loss, var_list=D_vars)
    G_optim = tf.train.AdamOptimizer(lr, beta1=0.4).minimize(G_loss, var_list=G_vars)
    DEC_optim = tf.train.AdamOptimizer(lr, beta1=0.4).minimize(DEC_loss, var_list=G_vars)

# train loop    
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    for epoch in range(0, 10):
        allIter = len(train_data) // batch_size
        for iter in range(len(train_data) // batch_size):
            
            x_ = train_data[iter*batch_size:(iter+1)*batch_size]
            message_ = (np.random.randint(2, size=[batch_size, MESSAGE_SIZE, 1, 1]) - 0.5) * 2
            
            # discriminator
            loss_d_, _ = sess.run([D_loss, D_optim], {x: x_, message: message_, isTrain: True})
            
            # generator
            message_ = (np.random.randint(2, size=[batch_size, MESSAGE_SIZE, 1, 1]) - 0.5) * 2
            loss_g_, _ = sess.run([G_loss, G_optim], {x: x_, message: message_, isTrain: True})

            # encoder
            message_ = (np.random.randint(2, size=[batch_size, MESSAGE_SIZE, 1, 1]) - 0.5) * 2
            loss_dec_, _ = sess.run([DEC_loss, DEC_optim], { x: x_, message: message_, isTrain: True})
        
            if (iter % 10 == 0):
                acc = test_message()
                show_result(epoch, iter, accuracy=str(round(acc,2)), path='results/' + str(epoch) + "_" + str(iter) + '.png')
                show_result(epoch, iter, accuracy=str(round(acc,2)), path='results/' + str(epoch) + "_" + str(iter) + '.png', binary=True)

                print(loss_dec_, loss_g_)
    

decode acc: 52.87500000000001%
2.042 1.296734
decode acc: 54.400000000000006%
0.941 4.1937194
decode acc: 49.425%
0.54 4.449348
decode acc: 49.675000000000004%
0.41 4.362099
decode acc: 50.525%
0.393 4.0969887
decode acc: 50.5%
0.509 4.3590565
decode acc: 49.4%
0.551 4.289226
decode acc: 50.475%
0.651 4.014549
decode acc: 52.225%
0.651 4.682645
decode acc: 55.35%
0.724 0.003653488
decode acc: 57.699999999999996%
0.595 4.38921
decode acc: 59.275%
0.674 3.8357947
decode acc: 64.60000000000001%
0.661 3.5692194
decode acc: 68.575%
0.565 1.6779939
decode acc: 75.875%
0.584 3.5696
decode acc: 81.1%
0.508 3.645909
decode acc: 81.675%
0.564 3.8120062
decode acc: 82.3%
0.589 0.69472045
decode acc: 85.85000000000001%
0.553 3.06124
decode acc: 87.85000000000001%
0.515 3.0571222
decode acc: 86.0%
0.58 3.6004179
decode acc: 85.275%
0.568 3.158241
decode acc: 85.82499999999999%
0.552 3.0979846
decode acc: 86.47500000000001%
0.562 3.65383
decode acc: 85.8%
0.562 3.4210644
decode acc: 84.05%
0.511 2.7

In [ ]:
MESSAGE_SIZE